In [1]:
import numpy as np
from pandas import Series, DataFrame
users = dict()
items = dict()
rows = []
cols = []
data = []
game_counts = [0] * 2437
user_counts = [0] * 6710
i = 0
j = 0
for l in open("drive/MyDrive/추천/train.json", 'r'):
  d = eval(l)
  u = d['userID']
  g = d['gameID']
  rating = d['hours_transformed']
  if u in users:
    ui = users[u]
  else:
    users[u] = i
    ui = i
    i += 1
  if g in items:
    gi = items[g]
  else:
    items[g] = j
    gi = j
    j += 1
  game_counts[gi] += 1
  user_counts[ui] += 1
  rows.append(ui)
  cols.append(gi)
  data.append(1)
for i in range(len(game_counts)):
  game_counts[i] = np.log(game_counts[i] + 1)
rows = np.array(rows)
cols = np.array(cols)
game_counts = np.array(game_counts)
# df= dict()
# df['userID'] = rows
# df['gameID'] = cols
# df = DataFrame(df)
# df['count'] = rating
with open("refined_train.csv", "w") as f2:
  f2.write("userID,gameID,counts\n")
  for i , j in zip(rows, cols):
    f2.write(f"{i},{j} , {1}\n")

In [2]:
with open("drive/MyDrive/추천/pairs_Played.csv", "r") as f1:
    f1.readline()
    lines = []
    uids = []
    gids = []
    g_cs = []
    u_cs = []
    p_ts = []
    test = dict()
    for line in f1:
        row_id, uid, gid = line.strip().split(",")
        lines.append(row_id)
        uids.append(users[uid])
        gids.append(items[gid])
    with open("Hours.csv", "w") as f2:
        f2.write("ID,userID,gameID\n")
        for a , b , c in zip(lines , uids , gids):
            f2.write(f"{a},{b},{c}\n")

In [3]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 14.1 MB/s eta 0:00:00


In [12]:
from scipy.sparse import csr_matrix
import numpy as np
rows = []
cols = []
data = []
with open("refined_train.csv" , "r") as f:
  f.readline()
  for line in f:
    uid , mid , _  = line.split(",")
    rows.append(int(uid))
    cols.append(int(mid))
    data.append(1)

rows = np.array(rows)
cols = np.array(cols)
data = np.array(data)

ratings_csr = csr_matrix((data , (rows , cols)))

from implicit import bpr

model = bpr.BayesianPersonalizedRanking(factors=10 , regularization=0.02)
model.fit(ratings_csr)


  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
with open("Hours.csv", "r") as f1:
  f1.readline()
  with open("result.csv", "w") as f2:
    f2.write("ID,Label\n")
    rows = []
    uids = []
    gids = []
    cnt = 10
    for line in f1:
      row_id, uid, gid = line.strip().split(",")
      uid = int(uid)
      gid = int(gid)
      x = [uid]
      ids , scores = model.recommend(x, ratings_csr[x] , N = 2437)#686
      ids = list(ids[0])
      s = scores[0]
      s = s + game_counts[ids]
      l = []
      for i , j in zip(s , ids):
        l.append((i , j))
      l.sort(key = lambda x : -x[0])
      answer = []
      for i in range(600):
        answer.append(l[i][1])
      if gid in answer:
        f2.write(f"{row_id},{1}\n")
      else:
        f2.write(f"{row_id},{0}\n")